In [117]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [118]:
# Complete project details at https://RandomNerdTutorials.com

try:
  import usocket as socket
except:
  import socket

from machine import Pin
import network

import esp
esp.osdebug(None)

import gc
gc.collect()



In [119]:
sta_if = network.WLAN(network.STA_IF)
ap_if = network.WLAN(network.AP_IF)
print('Station Up: ', sta_if.active())
print('Acess Point Up: ', ap_if.active())

Station Up:  False
Acess Point Up:  False


In [120]:
def connectX(typeX):
    typeX.active(True)
    print('Station Up: ', sta_if.active())
    print('Acess Point Up: ', ap_if.active())
    
def scanX():
    print('Status: %s\n' % sta_if.status())
    stations_list = sta_if.scan()
    for i, n in enumerate(stations_list, start=0):
        print('%s) %s' %(i, n))
    return stations_list
    

In [122]:
connectX(sta_if)
stations_list = []


if sta_if.active():
#     print('Station Up: ', sta_if.active())
    stations_list = scanX()
#     print(stations_list)
#     print(sta_if.status())
elif ap_if.active():
    print('Acess Point Up: ', ap_if.active())

Station Up:  True
Acess Point Up:  False
Status: 1000

0) (b'SM-A1kerr', b'\x8e\xf5\xa3\xa2\x7fn', 1, -43, 3, False)
1) (b'BPL_Wireless', b'\xe8\xed\xf3\xd8.0', 11, -50, 0, False)
2) (b'BPL_Wireless_2018', b'\xe8\xed\xf3\xd8.4', 11, -50, 0, False)
3) (b'BPL_Wireless_2018', b'4\xdb\xfd\n\x88\xa4', 1, -84, 0, False)
4) (b'BPL_Wireless', b'4\xdb\xfd\n\x88\xa0', 1, -85, 0, False)
5) (b'BPL_Wireless_2018', b',6\xf8\xb6#\xc4', 6, -85, 0, False)
6) (b'BPL_Wireless', b',6\xf8\xb6#\xc0', 6, -87, 0, False)
7) (b'BPLDAISY', b'\xb4u\x0e\xc2\xec\xe1', 1, -88, 3, False)


In [126]:
print('Connecting to: ', stations_list[0][0])
sta_if.connect(stations_list[0][0], '9054830760')
print('Status: %s\n' % sta_if.status())
print('Is connected: ', sta_if.isconnected())
print('Connected to: ', sta_if.config('essid'))
print('ifconfig: ', sta_if.ifconfig())


Connecting to:  b'SM-A1kerr'
Status: 1010

Is connected:  True
Connected to:  SM-A1kerr
ifconfig:  ('192.168.43.34', '255.255.255.0', '192.168.43.1', '192.168.43.1')


In [6]:
# Complete project details at https://RandomNerdTutorials.com

import socket
addr = socket.getaddrinfo('192.168.4.1', 80)[0][-1]
led = Pin(2, Pin.OUT)

def web_page():
  if led.value() == 1:
    gpio_state="ON"
  else:
    gpio_state="OFF"
  
  html = """<html><head> <title>ESP Web Server</title> <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" href="data:,"> <style>html{font-family: Helvetica; display:inline-block; margin: 0px auto; text-align: center;}
  h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display: inline-block; background-color: #e7bd3b; border: none; 
  border-radius: 4px; color: white; padding: 16px 40px; text-decoration: none; font-size: 30px; margin: 2px; cursor: pointer;}
  .button2{background-color: #4286f4;}</style></head><body> <h1>ESP Web Server</h1> 
  <p>GPIO state: <strong>""" + gpio_state + """</strong></p><p><a href="/?led=on"><button class="button">ON</button></a></p>
  <p><a href="/?led=off"><button class="button button2">OFF</button></a></p></body></html>"""
  return html

html2 = """<!DOCTYPE html>
<html>
    <head> <title>ESP8266 Pins</title> </head>
    <body> <h1>ESP8266 Pins</h1>
        <table border="1"> <tr><th>Pin</th><th>Value</th></tr> %s </table>
    </body>
</html>
"""

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# s.bind(('', 80))
s.bind(addr)
s.listen(5)

while True:
  conn, addr = s.accept()
  print('Got a connection from %s' % str(addr))
  request = conn.recv(1024)
  request = str(request)
  print('Content = %s' % request)
#   led_on = request.find('/?led=on')
#   led_off = request.find('/?led=off')
#   if led_on == 6:
#     print('LED ON')
#     led.value(1)
#   if led_off == 6:
#     print('LED OFF')
#     led.value(0)
#   response = web_page()
  conn.send('HTTP/1.1 200 OK\n')
  conn.send('Content-Type: text/html\n')
  conn.send('Connection: close\n\n')
#   conn.sendall(response)
  conn.close()

.......Got a connection from ('192.168.4.2', 36796)
Content = b''
Got a connection from ('192.168.4.2', 36797)
Content = b''
Got a connection from ('192.168.4.2', 36804)
Content = b'GET /?led=on HTTP/1.1\r\nHost: 192.168.4.1\r\nConnection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Linux; Android 8.0.0; SM-G930W8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.136 Mobile Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\r\nReferer: http://192.168.4.1/?led=on\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: en-GB,en-US;q=0.9,en;q=0.8,da;q=0.7\r\n\r\n'
LED ON
Got a connection from ('192.168.4.2', 36805)
Content = b'GET /?led=on HTTP/1.1\r\nHost: 192.168.4.1\r\nConnection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Linux; Android 8.0.0; SM-G930W8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.136 Mobile Safari/5

Traceback (most recent call last):
  File "<stdin>", line 39, in <module>
OSError: [Errno 104] ECONNRESET
